In [1]:
import sys
sys.path.append('D:/Grad School/PhD/AI/Final/Data/SumerSports')
import prep_data
import process_datasets
import models

#make sure to pip install polar and pytorch_lightning and pyarrow

In [2]:
import pandas as pd

plays_df = pd.read_csv('D:/Grad School/PhD/AI/Final/Data/input_data/plays.csv')

receiverA_counts =(plays_df['receiverAlignment']
    .value_counts()
    .sort_index()
    .to_frame()
)

print(receiverA_counts)
output_dimenstion = plays_df['receiverAlignment'].nunique()


                   count
receiverAlignment       
1x0                    8
1x1                  160
2x0                   45
2x1                 1824
2x2                 6483
3x0                    9
3x1                 6044
3x2                 1232
3x3                    1
4x1                  129
4x2                    1


In [3]:
prep_data.main()

Load players
Load plays
Load tracking
tracking_df rows: 6795800
Add features to tracking
Convert tracking to cartesian
Standardize play direction
Augment data by mirroring
Generate target - receiverAlignment
Split train/test/val
Total set: 3848 plays, 614734 frames
Train set: 2694 plays, 424714 frames
Test set: 576 plays, 94456 frames
Validation set: 578 plays, 95564 frames


In [ ]:
process_datasets.main()

Creating dataset for model_type='transformer', split='test'...


Pre-computing feature transforms: 100%|██████████| 94456/94456 [00:55<00:00, 1701.48it/s]


Took 2.6 mins
Creating dataset for model_type='transformer', split='val'...


Pre-computing feature transforms: 100%|██████████| 95564/95564 [00:55<00:00, 1716.92it/s]


Took 3.1 mins
Creating dataset for model_type='transformer', split='train'...


Pre-computing feature transforms: 100%|██████████| 424714/424714 [04:10<00:00, 1697.25it/s]


In [ ]:
import torch
from torch.utils.data import DataLoader
from process_datasets import load_datasets

train_dataset = load_datasets(model_type = 'transformer', split = 'train')
val_dataset = load_datasets(model_type = 'transformer', split = 'val')
test_dataset = load_datasets(model_type = 'transformer', split = 'test')

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 3)
val_loader = DataLoader(val_dataset, batch_size = batch_size, num_workers = 3)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, num_workers = 3)

for batch in train_loader:
    features, targets = batch
    print("Train features shape:", features.shape)
    print("Train targets shape:", targets.shape)
    break

In [ ]:
from models import SportsTransformerLitModel

feature_len = 5
model_dim = 64 #adjustable
num_layers = 4 #adjustable
dropout = 0.01
learning_rate = 1e-3
batch_size = 64
output_dim = plays_df['receiverAlignment'].nunique()

model = SportsTransformerLitModel(
    feature_len = feature_len,
    model_dim = model_dim,
    num_layers = num_layers,
    output_dim = output_dim,
    dropout = dropout,
    learning_rate = learning_rate,
)

In [ ]:
from pytorch_lighting import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pathlib import Path

checkpoint_callback = ModelCheckpoint(
    dirpath = Path("checkpoints/"),
    filename = "best-checkpoint",
    save_top_k = 1,
    verbose = True,
    monitor = "val_loss",
    mode = "min",
)

early_stop_callback = EarlyStopping(
    monitor = "val_loss",
    min_delta = 0.01,
    patience - 3,
    verbose = True,
    mode = "min"
)

trainer = Trainer(
    max_epochs = 20,
    accelerator = "gpu",
    devices = 1,
    callbacks = checkpoint_callback, early_stop_callback
)